In [30]:
import xmlschema
import pandas as pd
import os
import glob
import csv
import json
from IPython.display import Image, display
from pprint import pprint

In [31]:
APP_FOLDER='C:/Final/'
METADATA_FOLDER=APP_FOLDER+'Metadata/'
NEW_DATA_FOLDER=APP_FOLDER+'NewData/'
DATASET1_PATH=APP_FOLDER+'/SourceData/DataSet1/'
##for dataset 1
XMLSCHEMA_FILE=METADATA_FOLDER+'dataset1.xsd'

DATASET2_PATH=APP_FOLDER+'/SourceData/DataSet2/'
##

In [32]:
#get files list base on file type from a fodler
def get_file_list(file_path,file_type='xml'):
    return [ f for f in glob.glob(str(file_path)+'*.'+file_type)]


## Get csv file for dataset 1

### Generate csv file function

In [33]:
def get_image_metadata_csv(file_path,csv_file,xsd_file):
    xs = xmlschema.XMLSchema(xsd_file)
    file = open(csv_file, 'w+', newline ='') 
    csv_header=["filename","folder","segmented","depth","height","width",
                "object_id","xmax","xmin","ymax","ymin","difficult","name","occluded","truncated"]
    data=[csv_header]

    for f in get_file_list(file_path):
        imgs=xs.to_dict(f)
        imgs_head=[ imgs["filename"],imgs["folder"],imgs["segmented"],
                    imgs["size"]["depth"],imgs["size"]["height"],
                    imgs["size"]["width"]]
        i=0
        for o in imgs["object"]:
            imgs_fact=imgs_head.copy()
            imgs_fact.append(i)
            imgs_fact.append(o["bndbox"]["xmax"])
            imgs_fact.append(o["bndbox"]["xmin"])
            imgs_fact.append(o["bndbox"]["ymax"])
            imgs_fact.append(o["bndbox"]["ymin"])
            imgs_fact.append(o["difficult"])
            imgs_fact.append(o["name"])
            imgs_fact.append(o["occluded"])
            imgs_fact.append(o["truncated"])
            data.append(imgs_fact)
            i=i+1
    with file:     
        write = csv.writer(file) 
        write.writerows(data)

### generate dataset 1 csv file

In [34]:
csv_file_name_1=METADATA_FOLDER+'dataset1_images.csv'
get_image_metadata_csv(DATASET1_PATH+'/annotations/',csv_file_name_1,XMLSCHEMA_FILE)

## Get csv file for dataset 2

### Generate csv file function

In [35]:
def get_image_metadata_json_csv(json_file_path,csv_file,face_only=True):
    
    #prepare header data
    csv_header=["filename","folder","ID","tota_objects","object_id","xmax","xmin","ymax",
                "ymin","confidence","classname","isprotected"]
    csv_data=[csv_header]
    
    for file in get_file_list(json_file_path,'json'):
        with open(file) as f:
            data = json.load(f)
            filename=(data['FileName'])
            numofanno=int(data['NumOfAnno'])
            object_id=0
            total_objects=numofanno
            
            if face_only:
                total_objects=0
                for n in range(0,numofanno):
                    if ('face_with_mask'== data['Annotations'][n]['classname']
                       ) or('face_no_mask'==data['Annotations'][n]['classname']) :
                        total_objects=total_objects+1
            
            for n in range(0,numofanno):
                row=data['Annotations'][n]
                clname=row['classname']
                if(face_only and not ('face_with_mask' == clname)  and not ('face_no_mask' == clname)) :
                    continue
                imgs_fact=[filename]
                imgs_fact.append('images')
                imgs_fact.append(row['ID'])                
                imgs_fact.append(total_objects)
                imgs_fact.append(object_id)
                for i in range(0,4):
                    imgs_fact.append(int(row['BoundingBox'][i]))
                imgs_fact.append(row['Confidence'])
                imgs_fact.append(row['classname'])
                imgs_fact.append(row['isProtected'])
                csv_data.append(imgs_fact)
                object_id=object_id+1
    with open(csv_file, 'w', newline='') as file:    
        writer = csv.writer(file) 
        writer.writerows(csv_data)     

### generate dataset 2 csv file

In [36]:
json_file_folder=DATASET2_PATH+'Medical mask/Medical mask/Medical Mask/annotations/'
csv_file_name_2=METADATA_FOLDER+'dataset2_images.csv'
get_image_metadata_json_csv(json_file_folder,csv_file_name)

##### Combine dataset 1 and  dataset 2  to a new data set including croping face data 

In [38]:
import pandas as pd
from pathlib import Path
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

dataset1=pd.read_csv(csv_file_name_1)
dataset2=pd.read_csv(csv_file_name_2)

def get_label (label):
    w_mask=['face_with_mask','with_mask','mask_weared_incorrect']
    if (label in w_mask):
        return  'face_with_mask'
    else:
        return 'face_no_mask'

def get_face_mask_date(image_path, coords):
    image_obj = load_img(image_path)
#     print(image_path)
#     print(coords)
    cropped_image = image_obj.crop(coords)
    # newsize = (224, 224) 
    # cropped_image=cropped_image.resize(newsize)
    return cropped_image

def generate_new_dateset1(dataset):
#     new_data_folder='C:/Users/gerry/OneDrive/Documents/U of C/Project 607_608/NewData/'
#     absolute_path1=r'C:/Users/gerry/OneDrive/Documents/U of C/Project 607_608/Data/'
    for index, row in dataset.iterrows():
        try:
            label = get_label(row['name'])
            absolute_path=Path(DATASET1_PATH)
            img_file =Path.joinpath(absolute_path,row['folder'])
            img_file =Path.joinpath(img_file,row['filename'])
            image=get_face_mask_date(img_file,(row['xmin'],row['ymin'],row['xmax'],row['ymax']))
            image.save(NEW_DATA_FOLDER+label+'/'+Path(row['filename']).stem+'_'
                       +str(row['object_id'])+'_'+label+Path(row['filename']).suffixes[0])
        except  Exception as error:
            print(row['filename'])
            print((row['xmax'],row['xmin'],row['ymax'],row['ymin']))

def generate_new_dateset2(dataset):
#     new_data_folder='C:/Project 607_608/NewData/'
    absolute_path=DATASET2_PATH+'/Medical mask/Medical mask/Medical Mask/'
    for index, row in dataset.iterrows():
        try:
            label = row['classname']
            if(label!='face_with_mask' and label!='face_no_mask'):
                continue
            absolute_path=Path(absolute_path)
            img_file =Path.joinpath(absolute_path,row['folder'])
            img_file =Path.joinpath(img_file,row['filename'])
            image=get_face_mask_date(img_file,(row['xmax'],row['xmin'],row['ymax'],row['ymin']))
            image.save(new_data_folder+label+'/'+Path(row['filename']).stem+'_'
                        +str(row['object_id'])+'_'+label+Path(row['filename']).suffixes[0])
        except  Exception as error:
            print(row['filename'])
            print((row['xmax'],row['xmin'],row['ymax'],row['ymin']))

generate_new_dateset1(dataset1)
generate_new_dateset2(dataset2)


 #### Generate csv for the new data set

In [39]:
p = Path(NEW_DATA_FOLDER+'face_no_mask').glob('**/*')
p_w = Path(NEW_DATA_FOLDER+'face_with_mask').glob('**/*')
header = ["filename", "class_name"]
delimiter=','
with open("temp.csv", 'w') as f:
    writer = csv.writer(f,lineterminator='\n',delimiter=delimiter)
    writer.writerow(header)
    for f in p:
        p=Path(f)
        csv_row=[p.name,'face_no_mask']
        writer.writerow(csv_row)
    for f in p_w:
        p=Path(f)
        csv_row=[p.name,'face_with_mask']
        writer.writerow(csv_row)
        
df=pd.read_csv('temp.csv')
df=df.sample(frac=1)        
df.to_csv(METADATA_FOLDER+'new_data_set.csv',index=False)
os.remove("temp.csv")